In [1]:
import pandas as pd
import numpy as np
import json
import torch

# Direct imports from the same directory
from tabddpm_adapter import TabDDPMAdapter
from tabddpm_benchmark import evaluate_tabddpm, print_evaluation_results
from tabddpm_utils import preprocess_data, get_tstr_results

In [2]:
# Load configuration
print("# Load configuration")
with open("config.json", "r") as f:
    config = json.load(f)

# Load configuration


In [3]:
# 1. Load and prepare the dataset
print("# 1. Load and prepare the dataset")
data_raw = pd.read_csv("car.csv")
print(f"Columns in dataset: {data_raw.columns.tolist()}")
print(f"Dataset shape: {data_raw.shape}")
print(data_raw.head())

# 1. Load and prepare the dataset
Columns in dataset: ['Buying', 'Maint', 'Doors', 'Persons', 'Lug_boot', 'Safety', 'Class']
Dataset shape: (1728, 7)
  Buying  Maint Doors Persons Lug_boot Safety  Class
0  vhigh  vhigh     2       2    small    low  unacc
1  vhigh  vhigh     2       2    small    med  unacc
2  vhigh  vhigh     2       2    small   high  unacc
3  vhigh  vhigh     2       2      med    low  unacc
4  vhigh  vhigh     2       2      med    med  unacc


In [4]:
# 2. Preprocess data and detect categorical columns
print("\n# 2. Preprocess data and detect categorical columns")
data, categorical_columns = preprocess_data(data_raw)
print(f"Detected categorical columns: {categorical_columns}")

2025-04-27 10:45:44,183 - INFO - Converted Buying to category type (has 4 unique values)
2025-04-27 10:45:44,183 - INFO - Converted Maint to category type (has 4 unique values)
2025-04-27 10:45:44,187 - INFO - Converted Doors to category type (has 4 unique values)
2025-04-27 10:45:44,196 - INFO - Converted Persons to category type (has 3 unique values)
2025-04-27 10:45:44,197 - INFO - Converted Lug_boot to category type (has 3 unique values)
2025-04-27 10:45:44,204 - INFO - Converted Safety to category type (has 3 unique values)
2025-04-27 10:45:44,204 - INFO - Converted Class to category type (has 4 unique values)



# 2. Preprocess data and detect categorical columns
Detected categorical columns: ['Buying', 'Maint', 'Doors', 'Persons', 'Lug_boot', 'Safety', 'Class']


In [5]:
# 3. Define the target column for this dataset
print("\n# 3. Define the target column for this dataset")
target_column = "Class"
print(f"Target column: {target_column}")


# 3. Define the target column for this dataset
Target column: Class


In [6]:
# 4. Split the data into features and target
print("\n# 4. Split the data into features and target")
X = data.drop(columns=[target_column])
y = data[target_column]
print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"Target distribution:\n{y.value_counts()}")


# 4. Split the data into features and target
Features shape: (1728, 6)
Target shape: (1728,)
Target distribution:
Class
unacc    1210
acc       384
good       69
vgood      65
Name: count, dtype: int64


In [7]:
# 5. Initialize and train TabDDPM
print("\n# 5. Initialize and train TabDDPM")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tabddpm = TabDDPMAdapter(**config["tabddpm_params"], device=device)
print("Training TabDDPM model...")
tabddpm.fit(X, y)
print("Training completed")


# 5. Initialize and train TabDDPM
Training TabDDPM model...
Original data shape: (1728, 7), Target column: Class
Added OneHotEncoder for 6 categorical columns
Target 'Class' identified as categorical with 4 classes
Class mapping: {'acc': 0, 'good': 1, 'unacc': 2, 'vgood': 3}

Preprocessing Summary:
- Number of numerical features: 0
- Number of categorical features: 6
- Categorical columns: ['Buying', 'Maint', 'Doors', 'Persons', 'Lug_boot', 'Safety']
- Target column: Class
- Target type: Categorical
- Number of target classes: 4
X shape: (1728, 6), y shape: (1728,)
X_tensor shape: torch.Size([1728, 21]), y_tensor shape: torch.Size([1728])
X_tensor shape: torch.Size([1728, 21])
y_tensor shape: torch.Size([1728])


Epoch 299, Loss: 0.4677: 100%|██████████| 300/300 [01:29<00:00,  3.34it/s]

Training completed


In [8]:
# 6. Generate synthetic data
print("\n# 6. Generate synthetic data")
n_samples = 1000  
print(f"Generating {n_samples} synthetic samples...")
synthetic_data = tabddpm.generate(n_samples)
print(f"Generated {len(synthetic_data)} synthetic samples")
print("Synthetic data head:")
print(synthetic_data.head())


# 6. Generate synthetic data
Generating 1000 synthetic samples...
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Debug - out_dict type: <class 'torch.Tensor'>
Added target column 'Class' with 4 unique values
Final columns in synthetic data: ['Buying', 'Maint', 'Doors', 'Persons', 'Lug_boot', 'Safety', 'Class']
Generated 1000 synthetic samples
Synthetic data head:
  Buying  Maint  Doors Persons Lug_boot Safety  Class
0    low    low  5more       2      med   high   good
1  vhigh  vhigh      4    more    small    low   good
2  vhigh   high      4       4      med    low  unacc
3   high   high  5more       2      big    med  vgood
4    low    low  5more       2      big   high  vgood


In [9]:
# 7. Evaluate quality using TSTR and other metrics
print("\n# 7. Evaluate quality using TSTR and other metrics")
print("Running evaluation...")
evaluation_results = evaluate_tabddpm(data, synthetic_data, target_column=target_column)
print_evaluation_results(evaluation_results)



# 7. Evaluate quality using TSTR and other metrics
Running evaluation...


2025-04-27 10:48:25,672 - WARNING - No numerical columns found for likelihood fitness evaluation
2025-04-27 10:48:25,690 - INFO - Encoded categorical target with mapping: {'acc': 0, 'good': 1, 'unacc': 2, 'vgood': 3}
c:\Users\kamal\OneDrive\Documents\Vilo\DEAKIN\TRIMESTER 1 2025\SIT378 TEAM PROJECT B\KatabaticGitRepo\sit378\lib\site-packages\xgboost\core.py:158: UserWarning: [10:48:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
2025-04-27 10:48:29,872 - INFO - Encoded categorical targets with mapping: {'acc': 0, 'good': 1, 'unacc': 2, 'vgood': 3}
c:\Users\kamal\OneDrive\Documents\Vilo\DEAKIN\TRIMESTER 1 2025\SIT378 TEAM PROJECT B\KatabaticGitRepo\sit378\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached an

In [11]:
# 8. Extract and display TSTR results specifically
print("\n# 8. TSTR Performance Results")
tstr_results = get_tstr_results(evaluation_results)
if tstr_results is not None:
    print(tstr_results)


# 8. TSTR Performance Results
                    Accuracy        F1
LogisticRegression  0.325810  0.382918
RandomForest        0.244213  0.303822
MLP                 0.245370  0.304521
XGBoost             0.232639  0.292097


In [12]:
# 9. Save the synthetic data
print("\n# 9. Save synthetic data")
output_path = "car_synthetic.csv"
synthetic_data.to_csv(output_path, index=False)
print(f"Synthetic data saved to {output_path}")

print("\nTest completed successfully!")


# 9. Save synthetic data
Synthetic data saved to car_synthetic.csv

Test completed successfully!
